# VISUALIZATION

In [ ]:
import pandas as pd
import pickle
import plotly.graph_objects as go
from datetime import timedelta

## Load CSV files

In [ ]:

btc_df = pd.read_csv("Data/btc_df_saved.csv")
eth_df = pd.read_csv("Data/eth_df_saved.csv")
car_df = pd.read_csv("Data/car_df_saved.csv")
dog_df = pd.read_csv("Data/dog_df_saved.csv")

with open("Data/btc_garch_model.pkl", "rb") as f:
    btc_garch = pickle.load(f)

with open("Data/eth_garch_model.pkl", "rb") as f:
    eth_garch = pickle.load(f)    

with open("Data/car_garch_model.pkl", "rb") as f:
    car_garch = pickle.load(f)    

with open("Data/dog_garch_model.pkl", "rb") as f:
    dog_garch = pickle.load(f)

#### Plot Price Volatility


In [9]:
def plot_price_volatility(coins, days=90):
    
    fig = go.Figure()

    for name, df, garch in coins:
        df = df.copy()
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.dropna(subset=['Date']).sort_values('Date')
        df_recent = df[df['Date'] >= df['Date'].max() - timedelta(days=days)]

        # Map traces: (column name, y-axis, line style)
        traces = {
            f'{name} Price': ('Close', 'y1', 'solid'),
            f'{name} RealizedVol': ('RealizedVol', 'y2', 'solid'),
            f'{name} ATR': ('ATR', 'y2', 'dot'),
            f'{name} GARCH': ('GARCH', 'y2', 'dash') if garch is not None else None
        }

        for label, info in traces.items():
            if info and info[0] in df_recent.columns:
                y_values = garch[-len(df_recent):] if label.endswith('GARCH') and garch is not None else df_recent[info[0]]
                fig.add_trace(go.Scatter(
                    x=df_recent['Date'],
                    y=y_values,
                    mode='lines',
                    name=label,
                    yaxis=info[1],
                    line=dict(dash=info[2])
                ))

    fig.update_layout(
        title=f'Price and Volatility (Last {days} Days)',
        xaxis_title='Date',
        yaxis_title='Price',
        yaxis2=dict(title='Volatility', overlaying='y', side='right'),
        template='plotly_white',
        hovermode='x unified',
        width=1200,
        height=600
    )
    fig.show()



#### Result

In [10]:
coins = [("BTC", btc_df, btc_garch if 'btc_garch' in globals() else None)]
plot_price_volatility(coins, days=90)


In [8]:
coins = [("CAR", car_df, car_garch if 'car_garch' in globals() else None)]
plot_price_volatility(coins, days=90)


In [17]:
coins = [("ETH", eth_df, eth_garch if 'eth_garch' in globals() else None)]
plot_price_volatility(coins, days=90)


In [18]:
coins = [("DOG", dog_df, dog_garch if 'dog_garch' in globals() else None)]
plot_price_volatility(coins, days=90)
